# Generalized Linear Models for Binary Data (Lab1)

### Intro and objectives


### In this lab you will learn:
1. examples of generalized linear models
2. how to fit these models in Python


## What I hope you'll get out of this lab
* The feeling that you'll "know where to start" when you need to fit generalized linear models
* Worked Examples
* How to interpret the results obtained

In [1]:
!pip install wooldridge
!pip install linearmodels
import wooldridge as woo
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import linearmodels as plm
import numpy as np
from scipy import stats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 55.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 KB 3.5 MB/s eta 0:00:00


# Example. Married Women’s Labor Force Participation

#### Mroz (1987) conducts an analysis on 743 married women to estimate their incentives to join the labor force.

#### The response variable "inlf" is binary it takes value 1 if the woman works, it takes value 0 otherwise.

#### Different factors are considered: parner's income, level of education, experience, age, number of little children, number of children, etc.

In [5]:
MrozDataFrame=woo.dataWoo('mroz')


In [6]:
MrozDataFrame

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300.0,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300.0,0.6215,12,14,9.5,1,7,20.100058,1.524272,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,0,0,0,2,40,13,NaN,0.00,3020,43,...,28200.0,0.6215,10,10,9.5,1,5,28.200001,NaN,25
749,0,0,2,3,31,12,NaN,0.00,2056,33,...,10000.0,0.7715,12,12,7.5,0,14,10.000000,NaN,196
750,0,0,0,0,43,12,NaN,0.00,2383,43,...,9952.0,0.7515,10,3,7.5,0,4,9.952000,NaN,16
751,0,0,0,0,60,12,NaN,0.00,1705,55,...,24984.0,0.6215,12,12,14.0,1,15,24.983999,NaN,225


In [4]:
MrozDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   inlf      753 non-null    int64  
 1   hours     753 non-null    int64  
 2   kidslt6   753 non-null    int64  
 3   kidsge6   753 non-null    int64  
 4   age       753 non-null    int64  
 5   educ      753 non-null    int64  
 6   wage      428 non-null    float64
 7   repwage   753 non-null    float64
 8   hushrs    753 non-null    int64  
 9   husage    753 non-null    int64  
 10  huseduc   753 non-null    int64  
 11  huswage   753 non-null    float64
 12  faminc    753 non-null    float64
 13  mtr       753 non-null    float64
 14  motheduc  753 non-null    int64  
 15  fatheduc  753 non-null    int64  
 16  unem      753 non-null    float64
 17  city      753 non-null    int64  
 18  exper     753 non-null    int64  
 19  nwifeinc  753 non-null    float64
 20  lwage     428 non-null    float6

In [7]:
# Compute the correlation matrix
corr = MrozDataFrame[['inlf','nwifeinc','educ','exper','age','kidslt6','kidsge6']].corr()

round(corr,3)

,inlf,nwifeinc,educ,exper,age,kidslt6,kidsge6
inlf,1.000,-0.118,0.187,0.342,-0.080,-0.214,-0.002
nwifeinc,-0.118,1.000,0.278,-0.172,0.059,0.038,0.025
educ,0.187,0.278,1.000,0.066,-0.120,0.109,-0.059
exper,0.342,-0.172,0.066,1.000,0.334,-0.194,-0.300
age,-0.080,0.059,-0.120,0.334,1.000,-0.434,-0.385
kidslt6,-0.214,0.038,0.109,-0.194,-0.434,1.000,0.084
kidsge6,-0.002,0.025,-0.059,-0.300,-0.385,0.084,1.000


#### We observe weak, positive, levels of correlation between the response variable "inlf" and "exper". More experience seems to be correlated with a larger propensity to work.

#### We observe weak, negative, levels of correlation between the reponse variable and kidslt6. Having little children seems to be correlated with a lower propensity to work.

## 1. The model. 

#### We will assume that the random variable inlf follows a binomial distribution with parameters n and p 


$inlf\sim Binomial(n,p)$


#### We further assume a logit link function of the form:



$ logit(P(inlf=1))=log(\frac{P(inlf=1)}{P(inlf=0)})=\beta_0+\beta_1*nwifeinc+\beta_2*educ+\beta_3*exper+\beta_4*kidslt6+\beta_5*kidsge6 $

## 2. Estimation of the parameters
#### In this case we rely on MLS (OLS is no longer valid in non-linear contexts)

In [8]:
reg1 = smf.logit(formula='inlf ~ nwifeinc+educ+exper+age+kidslt6+kidsge6  ', data=MrozDataFrame)

# We fit the model
results1 = reg1.fit()
results1.summary()

Optimization terminated successfully.
         Current function value: 0.539367
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   inlf   No. Observations:                  753
Model:                          Logit   Df Residuals:                      746
Method:                           MLE   Df Model:                            6
Date:                Thu, 19 Jan 2023   Pseudo R-squ.:                  0.2112
Time:                        09:56:59   Log-Likelihood:                -406.14
converged:                       True   LL-Null:                       -514.87
Covariance Type:            nonrobust   LLR p-value:                 3.621e-44
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8379      0.841      0.996      0.319      -0.810       2.486
nwifeinc      -0.0202      0.008     -2.446      0.014      -0.036      -0.004
educ           0.2270      0.043      5.243      0.000       0.142       0.312
exper          0.1197      0.014      8.788      0.000       0.093       0.146
age           -0.0911      0.014     -6.361      0.000      -0.119      -0.063
kidslt6       -1.4394      0.201     -7.143      0.000      -1.834      -1.044
kidsge6        0.0582      0.073      0.793      0.428      -0.086       0.202
==============================================================================
"""

#### We add a quadratic term associated with experience

#### The final model is therefore:


$ log(\frac{P(inlf=1)}{P(inlf=0)})=0.83-0.020*nwifeinc+0.22*educ+0.11*exper-0.09*age-1.44*kidslt6+0.058*kidsge6$



## 3. Model interpretation:

#### All factors are statistically significant except kidsge6 (p-value: 0.428)


#### To interpret the estimates we must remember that a change in the independent variable changes the probability that the dependent variable takes the value 1 (in our case the probability of a woman to have a job).

#### For example the coefficient on educ means that, everything else held fixed, another year of education increases the log odds by 0.22

#### The coefficient on nwifeinc means that, everything else held fixed, another unit increase in the spouse's salary decrease the log odds by 0.020

#### The coefficient on kidslt6 means that, everything else held fixed, having another litte children decrease the log odss by 1.44


